# Research project for the understanding of actions in urban infrastructure and mapping of flood areas and landslides on slopes with the use of AI. 
 - Projeto pesquisa  para o entendimento das ações em infraestrutura urbana e mapeamentos de áreas de enchente e deslizamentos de enconstas com o uso de IA. 

# Clearing address from dataframe origin - Tratando base de dados dos endereços do dataframe original. 

In [ ]:
!pip install geopandas  

In [ ]:
!pip install shapely

In [ ]:
import matplotlib 
import pandas as pd
import geopandas as gpd

In [ ]:
from shapely.geometry import Point, Polygon  

In [ ]:
from geopandas import geoseries 

- csv file generated from building.ipynb notebook. -- Arquivo csv gerado a partir do notebook building.ipynb. 

In [ ]:
ppi_address_ajust = '/content/address_ppi.csv'
ppi_address_ajust = pd.read_csv(ppi_address_ajust, encoding='ISO-8859-1')
ppi_address_ajust.head()

In [ ]:
ppi_address_ajust.drop('Unnamed: 0', axis = 1)

In [ ]:
ppi_address_ajust.dtypes

In [ ]:
ppi_address_ajust.values

In [ ]:
ppi_address_ajust['ppi_ajust_cols'] = ppi_address_ajust['Empreendimento'] + '/' + ppi_address_ajust['UF']

In [ ]:
ppi_address_ajust

In [ ]:
pd.set_option('max_rows', 200)
pd.set_option('max_columns', 100)

- 'brazil_states' 

# Tools Geocode, for point location 
 - return GeometryCollection empty

In [ ]:
# gpd.tools.geocode(ppi_address_ajust['Empreendimento'], provider = 'nominatim', user_agent='UF Builds', country_bias = 'Brazil')

In [ ]:
# ppi_address_ajust['Geom'] = gpd.tools.geocode(ppi_address_ajust['Empreendimento'], provider = 'nominatim', user_agent='UF Builds', country_bias = 'brazil_states')['geometry']

In [ ]:
# ppi_address_ajust

In [ ]:
# ppi_address_ajust.loc[ppi_address_ajust['Geom'] == 'GEOMETRYCOLLECTION EMPTY', 'Geom'].head()

In [ ]:
# ppi_address_ajust.drop_duplicates() # if have, < inplace = True > 

# Geocoders implementation for locations acess - implementação de acesso das localizações usando o Geocoders 
- return None

In [ ]:
import geopy.geocoders

In [ ]:
from geopy.geocoders import Nominatim

In [ ]:
geopy.geocoders.options.default_user_agent = 'my_app/1'

In [ ]:
geopy.geocoders.options.default_timeout = 7

In [ ]:
geolocator = Nominatim()

In [ ]:
print(geolocator.headers)

{'User-Agent': 'my_app/1'}


In [ ]:
print(geolocator.timeout)

7


- Test aplication 

In [ ]:
# locator = Nominatim(user_agent='myGeocoder')
location = geolocator.geocode('Aeroporto de Salvador/ BA')

In [ ]:
print(location)

Aeroporto, Travessa Santos Dumont, Lauro de Freitas, Região Geográfica Imediata de Salvador, Região Metropolitana de Salvador, Região Geográfica Intermediária de Salvador, Bahia, Região Nordeste, 42700000, Brasil


In [ ]:
print(location.point)

12 54m 15.1844s S, 38 20m 27.9298s W


In [ ]:
location.point

Point(-12.904217899999999, -38.341091608139536, 0.0)

In [ ]:
from geopy.extra.rate_limiter import RateLimiter

- Search all address from dataframe

In [ ]:
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

In [ ]:
ppi_address_ajust['ppi_geocode'] = ppi_address_ajust['Empreendimento'].apply(geocode)

In [ ]:
ppi_address_ajust

- plot points of address 

In [ ]:
ppi_address_ajust['point'] = ppi_address_ajust['ppi_geocode'].apply(lambda loc: tuple(loc.point) if loc else None)

- ajust None's

In [ ]:
pd.set_option('max_rows', 200)
pd.set_option('max_columns', 100)

In [ ]:
ppi_address_ajust

In [ ]:
ppi_address_ajust.isna()

- New position from point values not null. 

In [ ]:
ppi_address_ajust.sort_values(by='point', ignore_index=True)

In [ ]:
ppi_address_ajust.index

RangeIndex(start=0, stop=191, step=1)

# Trait DF 

In [ ]:
ppi_point = ppi_address_ajust

In [ ]:
ppi_point

In [ ]:
ppi_point.sort_values(by='point', ignore_index=True)

In [ ]:
ppi_point = ppi_point.drop(['Unnamed: 0'], axis=1)

In [ ]:
ppi_point_true = ppi_point.iloc[:30, :3]

In [ ]:
ppi_point_true

In [ ]:
ppi_point_false = ppi_point.iloc[30:191, :3]

In [ ]:
ppi_point_false

In [ ]:
ppi_point_false.reset_index()

# individual search by address 
- Observar o conjunto de caracteres e a forma que se adeque a busa pelo geocode 

  

In [ ]:
location_00 = geolocator.geocode('BR-116 - Presidente Dutra	')

In [ ]:
print(location_00)

Rodovia Presidente Dutra, Cumbica, Jardim Presidente Dutra, Guarulhos, Região Imediata de São Paulo, Região Metropolitana de São Paulo, Região Geográfica Intermediária de São Paulo, São Paulo, Região Sudeste, 07180041, Brasil


In [ ]:
# ppi_point = ppi_address_ajust.drop(['Unnamed: 0',	'Empreendimento',	'UF',	'ppi_ajust_cols'], axis=1)

# Automate address lookup 
- Definir e selecionar uma forma de sanar os problemas que impedem a busca pela geoposição pelo Geocode por célula do dataframe para sanar as deficiências de preenchimento.

 - Softskill - string handlings

In [ ]:
import re 

In [ ]:
ppi_address_search_00 = ppi_point_false.iloc[0:160, :1]

- For find  condition
>>> 
- equivalent to applying < re.findall() > 
 - Series.str.findall(pat. flags=0) 



In [ ]:
ppi_address_search_00

In [ ]:
len(ppi_address_search_00)

160

In [ ]:
ppi_terminal = ppi_address_search_00[ppi_address_search_00['Empreendimento'].str.contains('Terminal')]

In [ ]:
ppi_terminal

In [ ]:
len(ppi_terminal)

29

In [ ]:
ppi_LT = ppi_address_search_00[ppi_address_search_00['Empreendimento'].str.contains('LT')]

In [ ]:
ppi_LT

In [ ]:
len(ppi_LT)

54

- Comparar e identificar as culunas

# To remember codes: 

In [ ]:
#show
# ppi_address_ajust.index.values
# ppi_address_ajust.Geom.values
# type(ppi_address_ajust.Geom.values)

# rename 
# ppi_address_ajust.rename(columns= {'Unnamed: 0': 'Rows_place'}, inplace=True )